In [1]:
# source https://www.nahuby.sk/tabulka_obdobia_rastu.php

In [1]:
import sys
!{sys.executable} -m pip install beautifulsoup4

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [3]:
obj = {
    'id': [],
    'type': [],
    'month': []
}

for i in range(1, 13):
    print('processing month', i)
    url = f"https://www.nahuby.sk/tabulka_obdobia_rastu.php?mode=&content=all&month={i}"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    mushrooms = soup.find('div', id="mainContent").find_all('div', class_='grid_3')
    arrays = []
    for m in mushrooms:
        brs = m.find_all('br')
        for br in brs:
            br.replace_with('\n')
        a = [s.strip() for s in m.text.split("\n") if s.strip() != '']
        if len(a) == 2:
            name = a[0]
            type_ = a[1]
        if len(a) == 3:
            name = a[0]
            type_ = a[2]
        if len(a) == 4:
            name = a[0]
            secondary_name = a[1]
            if name == '-':
                name = secondary_name
            type_ = a[3]

        obj['id'].append(name)
        obj['type'].append(type_)
        obj['month'].append(i)
    
len(obj['id'])

processing month 1
processing month 2
processing month 3
processing month 4
processing month 5
processing month 6
processing month 7
processing month 8
processing month 9
processing month 10
processing month 11
processing month 12


9552

In [6]:
types = {'vzácna huba', 'nejedlá huba', 'výborná jedlá huba', 'jedovatá huba', 'chránená huba'}

def transform_type(t):
    if t not in types:
        return 0
    if t == 'výborná jedlá huba' or t == 'jedlá huba':
        return 1
    return 0

df = pd.DataFrame(obj).drop_duplicates()
# df.drop_duplicates()
df['type'] = df['type'].str.lower()
df['edible'] = df['type'].apply(lambda x: transform_type(x))
df['id'] = df['id'].str.capitalize()
df = df[df['id'] != '-']
df

,id,type,month,edible
0,Lilapila jurana,vzácna huba,1,0
1,Amphosoma atroolivaceum,vzácna huba,1,0
2,Hyalorbilia helicospora,nejedlá huba,1,0
3,Hyalorbilia subfusispora,nejedlá huba,1,0
4,Helicogonium orbiliarum,nejedlá huba,1,0
...,...,...,...,...
9547,Xenazma veľkovýtrusná,-,12,0
9548,Žezlovka,vzácna huba,12,0
9549,Žezlovka hmyzová,nejedlá huba,12,0
9550,Žezlovka mravcová,vzácna huba,12,0


In [7]:
aggregated = df[['id', 'month', 'edible']].groupby(['id', 'edible']).agg({'month': ['min', 'max']}).reset_index()
aggregated.columns = ['id', 'edible', 'start', 'end']
aggregated = aggregated[['id', 'start', 'end', 'edible']]
aggregated['type'] = 3 # mushroom
aggregated

,id,start,end,edible,type
0,Amphosoma atroolivaceum,1,12,0,3
1,Antrodiela buková,7,11,0,3
2,Antrodiela lopatkovitá,7,11,0,3
3,Antrodiela podobná,1,12,0,3
4,Antrodiela žltooranžová,7,11,0,3
...,...,...,...,...,...
1779,Žezlovka veľkovýtrusná,8,11,0,3
1780,Žilnačka belavá,6,11,0,3
1781,Žilnačka lindtnerova,6,11,0,3
1782,Žilnačka oranžová,8,11,0,3


In [8]:
aggregated.to_csv('./mushrooms.csv')

In [9]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(aggregated[aggregated['edible'] == 1])

                              id  start  end  edible  type
59               Bedľa vyniknutá      7    9       1     3
60                  Bedľa vysoká      7   10       1     3
61      Bedľa vysoká čiernohnedá      8   10       1     3
191                 Hliva buková      4   11       1     3
192                 Hliva dubová      9   11       1     3
196            Hliva ustricovitá      1   12       1     3
265                 Hríb brezový      8   10       1     3
266                Hríb bronzový      5   10       1     3
267                  Hríb dubový      5   11       1     3
272                 Hríb hrabový      6   11       1     3
288                    Hríb siný      5   10       1     3
290                Hríb smrekový      8   11       1     3
291     Hríb smrekový kyjakovitý      6   11       1     3
292                 Hríb sosnový      5   11       1     3
298         Hríb zrnitohlúbikový      6   11       1     3
304            Hríbnik modrejúci      6   11       1    